In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.cluster.vq import kmeans
from scipy.cluster.vq import vq
from scipy.stats import norm
from multiprocessing import Pool
from pathlib import Path
import json
import os
from experiment_utils import Experiment, ExperimentFilter, Plotting

In [ ]:
nominal_speed_ghz = 2.0
ACCESS_LATENCY_NS = 'Core to core communication latency [ns]'
MIN_MU_ACCESS_LATENCY_NS = 'Minimal core to core communication latency [ns]'
MU_ACCESS_LATENCY_NS = 'Average core to core communication latency [ns]'
MAX_MU_ACCESS_LATENCY_NS = 'Maximal core to core communication latency [ns]'
STD_ACCESS_LATENCY_NS = 'Stddev of core to core communication latency [ns]'
DIFF_MU_ACCESS_LATENCY_NS = 'Difference in core to core communication latency [ns]'
CORE_FREQUENCY = 'Core frequency [MHz]'
UNCORE_FREQUENCY = 'Unore frequency [MHz]'
PING_CORE = 'Ping CPU with memory allocated in its NUMA node'
PONG_CORE = 'Pong CPU'
num_cacheboxes = 14

In [ ]:
def get_latencies(latencies_results_txt: str):
    def mu(data):
        return norm.fit(data)[0]

    def std(data):
        return norm.fit(data)[1]

    data_all = pd.read_csv(latencies_results_txt)
    # divide by 2 as we do two accesses with each ping pong
    data_all['LatencyPerAccess'] = data_all['Latency'] / data_all['Repetitions Set'] / 2
    data_all[ACCESS_LATENCY_NS] = data_all['LatencyPerAccess'] / nominal_speed_ghz

    cachelines_modeled = pd.pivot_table(data_all, values=ACCESS_LATENCY_NS, index='Cache Line', aggfunc={ACCESS_LATENCY_NS: [mu, std]})
    cachelines_modeled = cachelines_modeled.rename(columns={'mu': MU_ACCESS_LATENCY_NS, 'std': STD_ACCESS_LATENCY_NS})

    sorted_cachelines = sorted(cachelines_modeled[MU_ACCESS_LATENCY_NS])

    min_points = [ 1/num_cacheboxes/2 + i/num_cacheboxes for i in range(num_cacheboxes) ]
    k_points = [ sorted_cachelines[int(percentatge * len(cachelines_modeled))] for percentatge in min_points ]

    codebook, _distortion = kmeans(cachelines_modeled[MU_ACCESS_LATENCY_NS], k_points)

    code, _distortion = vq(cachelines_modeled[MU_ACCESS_LATENCY_NS], codebook)

    slice_latencies = list()
    for i in range(num_cacheboxes):
        mu, std = norm.fit(cachelines_modeled[MU_ACCESS_LATENCY_NS][code == i])
        slice_latencies.append(mu)
        # print(f'Cache Slice 0: Mu: {mu} Std: {std}')
    slice_latencies = sorted(slice_latencies)

    formated_slice_latencies = list(map(lambda v: "{:.1f}".format(v), slice_latencies))
    # print(f'Latencies of each of the {num_cacheboxes} cache slices: {formated_slice_latencies}')

    return cachelines_modeled, slice_latencies


In [ ]:
def plot_single_core_pair(cachelines_modeled, slice_latencies, vmin, vmax):
    buckets = [ 100*i/num_cacheboxes for i in range(num_cacheboxes) ]

    ax = sns.ecdfplot(cachelines_modeled, x=MU_ACCESS_LATENCY_NS, stat='percent')
    ax.set_xlim(vmin, vmax)
    ax.set_ylim(0, 100)

    for i in buckets:
        plt.axhline(y=i, color='black', dashes=[2], linewidth=1)

    for i in slice_latencies:
        plt.axvline(x=i, color='black', dashes=[2], linewidth=1)

In [ ]:
experiments = Experiment.get_experiments()
experiments = list(filter(ExperimentFilter.by_experiment_name('core-to-core-latency'), experiments))
experiment = ExperimentFilter.get_latest(experiments)
experiment

In [ ]:
cachelines_modeled, slice_latencies = get_latencies(f'{experiment.path}/3800000/25/0/1/latency_results.txt')
plot_single_core_pair(cachelines_modeled, slice_latencies, vmin=35, vmax=50)
Plotting.savefig(experiment, 'core0to1latency.pdf', annotations_y_offset=0, annotations_y_spacing=0.025)

In [ ]:
def mean(list_int):
    return sum(list_int) / len(list_int)

In [ ]:
def get_dataframe_from_file(file):
    file_path = Path(file)
    data_path = file_path.parent / 'settings.json'

    with open(data_path, 'r') as f:
        data = json.load(f)

        try:
            _cachelines_modeled, slice_latencies = get_latencies(file)
        except:
            return None

        data[PING_CORE] = data['this_cpu']
        data[PONG_CORE] = data['other_cpu']
        data[MIN_MU_ACCESS_LATENCY_NS] = min(slice_latencies)
        data[MU_ACCESS_LATENCY_NS] = mean(slice_latencies)
        data[MAX_MU_ACCESS_LATENCY_NS] = max(slice_latencies)
        data[DIFF_MU_ACCESS_LATENCY_NS] = max(slice_latencies) - min(slice_latencies)
        data[CORE_FREQUENCY] = data['core_frequency'] // 1000
        data[UNCORE_FREQUENCY] = data['uncore_frequency'] * 100

        return pd.DataFrame([data])

In [ ]:
def read_files(files) -> pd.DataFrame:
    with Pool(os.cpu_count()) as p:
        optional_dataframe_list = p.map(get_dataframe_from_file, files)

        data = pd.DataFrame()

        for entry in optional_dataframe_list:
            if entry is None:
                continue
            data = pd.concat([
                data,
                entry
            ])

        return data

In [ ]:
files = experiment.path.rglob('latency_results.txt')
data = read_files(files)
data

In [ ]:
vmin = min(data[MIN_MU_ACCESS_LATENCY_NS])
vmin = vmin//10*10
vmin

In [ ]:
vmax = min(data[MAX_MU_ACCESS_LATENCY_NS])
vmax = (vmax//10+1)*10
vmax

In [ ]:
def get_min_max(data):
    vmin = min(data[MIN_MU_ACCESS_LATENCY_NS])
    vmin = vmin//10*10

    vmax = min(data[MAX_MU_ACCESS_LATENCY_NS])
    vmax = (vmax//10+1)*10

    return vmin, vmax

In [ ]:
def plot_core_to_core_heatmap(heatmap_data, metric, **kwargs):
    fig, ax = plt.subplots(figsize=(10, 10))

    sns.heatmap(heatmap_data.pivot(index=PING_CORE, columns=PONG_CORE, values=metric),
                ax=ax,
                square=True,
                cmap="viridis",
                linecolor="black",
                linewidths=0.5,
                cbar_kws={'label': metric},
                **kwargs)

    core_frequencies = heatmap_data[CORE_FREQUENCY].unique()
    uncore_frequencies = heatmap_data[UNCORE_FREQUENCY].unique()

    assert(len(core_frequencies) == 1)
    assert(len(uncore_frequencies) == 1)

    ax.text(x=0.5,
            y=1.05,
            s=f'Core to core latency on socket 0 with {core_frequencies[0]}MHz core and {uncore_frequencies[0]}MHz uncore frequency',
            va="top",
            ha="center",
            transform=ax.transAxes)

In [ ]:
all_core_data = pd.DataFrame()

for core_frequency in data[CORE_FREQUENCY].unique():
    for uncore_frequency in data[UNCORE_FREQUENCY].unique():
        filtered_data = data[data[CORE_FREQUENCY] == core_frequency]
        filtered_data = filtered_data[filtered_data[UNCORE_FREQUENCY] == uncore_frequency]

        all_core_data = pd.concat([
            all_core_data,
            pd.DataFrame([{
                CORE_FREQUENCY: core_frequency,
                UNCORE_FREQUENCY: uncore_frequency,
                MIN_MU_ACCESS_LATENCY_NS: min(filtered_data[MIN_MU_ACCESS_LATENCY_NS]),
                MAX_MU_ACCESS_LATENCY_NS: max(filtered_data[MAX_MU_ACCESS_LATENCY_NS]),
                DIFF_MU_ACCESS_LATENCY_NS: max(filtered_data[DIFF_MU_ACCESS_LATENCY_NS]),
            }])
        ])

        vmin, vmax = get_min_max(filtered_data)

        plot_core_to_core_heatmap(filtered_data, MIN_MU_ACCESS_LATENCY_NS, vmin=vmin, vmax=vmax)
        Plotting.savefig(experiment, f'core-to-core-heatmap-min-{core_frequency}-{uncore_frequency}.pdf', annotations_y_offset=0.1, annotations_y_spacing=0.0125, annotations_x_offset=0.1)

        plot_core_to_core_heatmap(filtered_data, MU_ACCESS_LATENCY_NS, vmin=vmin, vmax=vmax)
        Plotting.savefig(experiment, f'core-to-core-heatmap-avg-{core_frequency}-{uncore_frequency}.pdf', annotations_y_offset=0.1, annotations_y_spacing=0.0125, annotations_x_offset=0.1)

        plot_core_to_core_heatmap(filtered_data, MAX_MU_ACCESS_LATENCY_NS, vmin=vmin, vmax=vmax)
        Plotting.savefig(experiment, f'core-to-core-heatmap-max-{core_frequency}-{uncore_frequency}.pdf', annotations_y_offset=0.1, annotations_y_spacing=0.0125, annotations_x_offset=0.1)

        plot_core_to_core_heatmap(filtered_data, DIFF_MU_ACCESS_LATENCY_NS)
        Plotting.savefig(experiment, f'core-to-core-heatmap-diff-{core_frequency}-{uncore_frequency}.pdf', annotations_y_offset=0.1, annotations_y_spacing=0.0125, annotations_x_offset=0.1)


In [ ]:
all_core_data

In [ ]:
def plot_all_core_heatmap(heatmap_data, metric, **kwargs):
    fig, ax = plt.subplots(figsize=(5, 5))

    sns.heatmap(heatmap_data.pivot(index=UNCORE_FREQUENCY, columns=CORE_FREQUENCY, values=metric),
                ax=ax,
                square=True,
                annot=True,
                cmap="viridis",
                linecolor="black",
                linewidths=0.5,
                cbar_kws={'label': metric},
                fmt=".1f",
                **kwargs)

In [ ]:
plot_all_core_heatmap(all_core_data, MIN_MU_ACCESS_LATENCY_NS)
Plotting.savefig(experiment, f'all-to-all-heatmap-min.pdf', annotations_y_offset=0.1, annotations_y_spacing=0.025, annotations_x_offset=0.01)

plot_all_core_heatmap(all_core_data, MAX_MU_ACCESS_LATENCY_NS)
Plotting.savefig(experiment, f'all-to-all-heatmap-max.pdf', annotations_y_offset=0.1, annotations_y_spacing=0.025, annotations_x_offset=0.01)

plot_all_core_heatmap(all_core_data, DIFF_MU_ACCESS_LATENCY_NS)
Plotting.savefig(experiment, f'all-to-all-heatmap-diff.pdf', annotations_y_offset=0.1, annotations_y_spacing=0.025, annotations_x_offset=0.01)